# Presidential History
In this Notebook, we're taking the presidential-votes table and turning it into a presidential history table, where
we map the party histories by state and year.  The resulting table will have columns (State, Year, Republican, Democratic, Libertarian, Other), where the latter four columns give the percentage of the vote of each of the parties in that election.  This shows the value of interoperation of SDML/SDQL with Python's native structures and PANDAS dataframes.

In [ ]:
import json
import sdtp
import pandas as pd


In [ ]:
with open('tables/presidential-elections.sdml') as f:
  desc = json.load(f)
  table = sdtp.TableBuilder.build_table(desc)


We want the parties to be (Democratic, Republican, Libertarian, and Other) for the Presidential history.  So we're going to replace every other value with 'Other'. 

In [ ]:
final_parties = ['Democratic', 'Republican', 'Libertarian']
for row in table.rows:
  # Canonical_Party is column 2
  row[2] = row[2] if row[2] in final_parties else 'Other'

Get rid of the Nationwide vote, and only keep the 'Canonical_Party', 'State', 'Year', and 'Percentage' columns

In [ ]:
states_only = {"operator": "NONE", "arguments": [{"operator": "IN_LIST", "column": "State", "values": ["Nationwide"]}]}
filtered = table.get_filtered_rows(states_only, columns=['Canonical_Party', 'State', 'Year', 'Percentage'], format='sdml')

Form the table as a pivot table using PANDAS

In [ ]:
df = filtered.to_dataframe()
elections_by_party = df.pivot_table(
    index=['Year', 'State'],
    columns='Canonical_Party',
    values='Percentage',
    fill_value=0  # Optional: fill missing with 0, or use None if you prefer
).reset_index()
elections_by_party

Get an SDML Table from the DataFrame

In [ ]:
column_names = elections_by_party.columns.tolist()
typelist = ['string', 'string', 'number', 'number', 'number', 'number']
schema = [{"name": column_names[i], "type": typelist[i]} for i in range(len(column_names))]
result = sdtp.RowTable(schema, elections_by_party.values.tolist())

Save it

In [ ]:
save_form = result.to_dictionary()
with open('scratch/presidential-history.sdml', 'w') as f:
  json.dump(save_form, f, indent=2)
